# Describe VTK conversion here

In [31]:
from pathlib2 import Path
import meshio
from eq_fault_geom import faultmeshio # this could be added to meshio ??\
import geopandas as gpd

## Convert Hi-res fault surfaces

These are the CFM MOVE tsurf files, converted into VTK format

In [32]:
#convert the tsurf files from CFM into VTK format
inpath = Path("./data/CFM/tsurf")
outpath = Path("./tmp/hi-res")

#ensure outpath exists
outpath.mkdir(parents=True, exist_ok=True)

#iterate over the *.ts files, sor
for tsurfFile in inpath.glob('*.ts'):
    outFile = Path.joinpath(outpath, tsurfFile.stem).with_suffix('.vtk')
    tsurf = faultmeshio.tsurf(tsurfFile)
    meshio.write(str(outFile), tsurf.mesh, binary=False)

## Convert stirling surfaces as used by the opensha-*

These are created from the 2D Shapefiles in CFM

In [33]:
# Read in data
shp_df = gpd.GeoDataFrame.from_file("zip://./data/CFM/GIS/NZ_CFM_v0_3_180620.zip!NZ_CFM_v0_3_180620.shp")

# Convert to NZTM
#shp_df.to_crs(epsg=2193)

In [34]:

import cfm_shp_to_vtk as cv

In [35]:
flt1 = shp_df.FZ_Name.str.contains("Wellington Hutt Valley")
flt2 = shp_df.FZ_Name.str.contains("Wairarapa")
faults = shp_df[flt2|flt1]

faults = faults.sort_values("FZ_Name").reset_index(drop=True)
sorted_wgs = faults.to_crs(epsg=4326)

cv.output_path = Path("./tmp/lo-res")
cv.output_path.mkdir(parents=True, exist_ok=True)

for i, fault in sorted_wgs.iterrows():
    nztm_geometry_i = faults.iloc[i].geometry
    cv.create_stirling_vtk(fault, section_id=i, nztm_geometry=nztm_geometry_i)
